In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
from tensorflow.keras.optimizers import RMSprop


#### Pré processamento das Imagens

In [3]:
import os

In [4]:
local = '/content/drive/MyDrive/23.1 Visão Computacional/Trabalho final/Treinamento/Treinamento Classificação'

In [5]:
dir = list(os.listdir(local))
dir

['Fumaca', 'Nao_Fumaca']

In [6]:
print(f'{dir[0]} contem :', len(os.listdir(f'{local}/{dir[0]}')),'fotos')
print(f'{dir[1]} contem :', len(os.listdir(f'{local}/{dir[1]}')),'fotos')

size = sum([len(os.listdir(f'{local}/{dir[0]}')) ,len(os.listdir(f'{local}/{dir[1]}'))])
print('Total:', size)

Fumaca contem : 756 fotos
Nao_Fumaca contem : 764 fotos
Total: 1520


In [7]:
amostras = int(size/8)
amostras


190

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Para reescalar as imagens em 1./255  (normalizar)
reescala = ImageDataGenerator(rescale=1/255)


imagens_treinamento = reescala.flow_from_directory(
        local,
        target_size=(224, 24),  # Reescala as imagens para o input da VGG
        batch_size= amostras,
        class_mode='binary')

Found 1520 images belonging to 2 classes.


#### Transfer Learning

In [9]:
# importa o modelo vgg16 pre treinado

In [10]:
vgg = tf.keras.applications.vgg16.VGG16(include_top = False, weights= 'imagenet', pooling= 'max', input_shape = (224,224,3))

In [11]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [12]:
# opcional para congelar o treinamento das camadas de convolução (False)

vgg.trainable = True

In [13]:
flatten_layer = layers.Flatten()(vgg.output)

dense_layer = layers.Dense(1024, activation = 'relu')(flatten_layer)

output_layer = layers.Dense(1, activation = 'sigmoid')(dense_layer)

In [14]:
model = models.Model(vgg.input, output_layer)       # cria o modelo com a vgg de input (camadas de convolucao) e a output layer definida acima

In [15]:
model.compile(optimizer = RMSprop(learning_rate = 0.0001),
                                  loss = 'binary_crossentropy',
                                  metrics = ['acc'])

#### Treinamento

In [ ]:
history = model.fit(
      imagens_treinamento,
      steps_per_epoch=8,  # total de x imagens e cada batch com y imagens , logo o passo será x/y
      epochs=10,
      verbose=2)

Epoch 1/10
